In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data_dir = "~/Japanese_future_real_estate_prices/Data/2005_2013/trade_prices"
# Look at 01 - Hokkaido#

df_01 = pd.read_csv(f'{data_dir}/01.csv')

df_01.info()
df_01.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186238 entries, 0 to 186237
Data columns (total 38 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   No                           186238 non-null  int64  
 1   Type                         186238 non-null  object 
 2   Region                       133069 non-null  object 
 3   MunicipalityCode             186238 non-null  int64  
 4   Prefecture                   186238 non-null  object 
 5   Municipality                 186238 non-null  object 
 6   DistrictName                 183567 non-null  object 
 7   NearestStation               145929 non-null  object 
 8   TimeToNearestStation         144490 non-null  object 
 9   MinTimeToNearestStation      144490 non-null  float64
 10  MaxTimeToNearestStation      140917 non-null  float64
 11  TradePrice                   186238 non-null  int64  
 12  FloorPlan                    20486 non-null   object 
 13 

/tmp/ipykernel_166860/2007917348.py:4: DtypeWarning: Columns (12,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_01 = pd.read_csv(f'{data_dir}/01.csv')


,No,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,...,Classification,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks
0,1,Residential Land(Land Only),Residential Area,1101,Hokkaido,"Chuo Ward,Sapporo City",Asahigaoka,Maruyamakoen,28,28.0,...,City Road,10.2,Category I Exclusively Low-story Residential Zone,40.0,80.0,4th quarter 2018,2018,4,NaN,NaN
1,2,Residential Land(Land Only),Residential Area,1101,Hokkaido,"Chuo Ward,Sapporo City",Asahigaoka,Maruyamakoen,27,27.0,...,City Road,7.2,Category I Exclusively Low-story Residential Zone,40.0,80.0,4th quarter 2018,2018,4,NaN,NaN
2,3,"Pre-owned Condominiums, etc.",NaN,1101,Hokkaido,"Chuo Ward,Sapporo City",Asahigaoka,Maruyamakoen,20,20.0,...,NaN,NaN,Neighborhood Commercial Zone,80.0,200.0,4th quarter 2018,2018,4,Not yet,NaN
3,4,Residential Land(Land Only),Residential Area,1101,Hokkaido,"Chuo Ward,Sapporo City",Asahigaoka,Maruyamakoen,26,26.0,...,Private Road,4.0,Category I Exclusively Low-story Residential Zone,40.0,80.0,2nd quarter 2018,2018,2,NaN,NaN
4,5,Residential Land(Land Only),Residential Area,1101,Hokkaido,"Chuo Ward,Sapporo City",Asahigaoka,Maruyamakoen,23,23.0,...,City Road,7.2,Category I Exclusively Low-story Residential Zone,40.0,80.0,2nd quarter 2018,2018,2,NaN,NaN


# My Initial Thoughts/Comments
## Useful

 1.   **Type**                         186238 non-null  object  : This is a string explaining the type of real estate. Useful but maybe hard to work with. Can filter land v building with this.
 
 2.   **Region**                       133069 non-null  object  : Type of area i.e. Commercial / residential / Industrial. Useful but maybe hard to work with.

 4.   **Prefecture**                   186238 non-null  object  : Useful. Probably need to encode. 

 5.   **Municipality**                 186238 non-null  object  : Useful. Can I encode with the municipality code?
 
 9.   **MinTimeToNearestStation**      144490 non-null  float64 : Useful.

 10.  **MaxTimeToNearestStation**      140917 non-null  float64 : Useful. Consider taking mean with previous value?

 11.  **TradePrice**                   186238 non-null  int64   : Useful.

 12.  **FloorPlan**                    20486 non-null   object  : Useful. Looks difficult to work with.

 13.  **Area**                         186238 non-null  int64   : Useful.

 14.  **AreaIsGreaterFlag**            186238 non-null  int64   : Useful. Flag if area is larger than 2000 square meters. Convert to bool?

 18.  **Frontage**                     125257 non-null  float64 : Useful.

 19.  **FrontageIsGreaterFlag**        186238 non-null  bool    : Useful. Flag if length of frontage greater than 50m

 20.  **TotalFloorArea**               62490 non-null   float64 : Useful.

 21.  **TotalFloorAreaIsGreaterFlag**  186238 non-null  int64   : Useful. Floor area greater than 2000 sqyare meters. Convert to bool?

 22.  **BuildingYear**                 81078 non-null   float64 : Useful.

 23.  **PrewarBuilding**               186238 non-null  int64   : Useful. True if building year < 1945. Convert to bool?

 25.  **Use**                          81249 non-null   object  : Not so useful for price prediction but useful for filtering data for (non)residential buildings.

 26.  **Purpose**                      56882 non-null   object  : Not so useful for price prediction but useful for filtering data for (non)residential buildings.

 34.  **Year**                         186238 non-null  int64   : Useful for timing trends.

 35.  **Quarter**                      186238 non-null  int64   : Useful for timing trends.


 ## Probably not useful

 **No** - Just an Identifier

 3.   **MunicipalityCode**             186238 non-null  int64   : City code of Japan.

  6.   **DistrictName**                 183567 non-null  object  : Less important than Municipality and Prefecture.

 7.   **NearestStation**               145929 non-null  object  : Probably matters more if there is a station or not, not which station it is.

 8.   **TimeToNearestStation**         144490 non-null  object  : Messy data. Use min / max values instead.

 15.  **UnitPrice**                    66260 non-null   float64 : Land price per square meter. Redundant since we care about the total trade price. If this is land only or not info is available in Type col.  

 16.  **PricePerTsubo**                66260 non-null   float64 : Probably made redundant given area and tradeprice. 

 17.  **LandShape**                    132674 non-null  object  : Assumed to be unimportant to first order.

 24.  **Structure**                    81430 non-null   object  : Assumed to be unimportant to first order.

 27.  **Direction**                    132631 non-null  object  : Assumed to be unimportant to first order.

 28.  **Classification**               129636 non-null  object  : Claddification of outside road. Probably redundant from region information.

 29.  **Breadth**                      128012 non-null  float64 : Breadth of outside road. Probably redundant from region information.

 30.  **CityPlanning**                 152003 non-null  object  : Assumed to be unimportant to first order.

 31.  **CoverageRatio**                140562 non-null  float64 : Assumed to be unimportant to first order.

 32.  **FloorAreaRatio**               140562 non-null  float64 : Assumed to be unimportant to first order.

 33.  **Period**                       186238 non-null  object  : Redundant by Year and Quarter Columns.

 36.  **Renovation**                   19731 non-null   object  : Maybe important but data is sparse.
 
 37.  **Remarks**                      6550 non-null    object  : Sparse. Likely irrelavant in the majority of cases.


In [3]:
# Remove unwanted columns #
df_01.drop(columns=['No', 'FloorPlan', 'Remarks', 'Renovation', 
                     'LandShape', 'Structure', 'Direction', 'Classification', 'Breadth', 
                     'CityPlanning', 'CoverageRatio', 'FloorAreaRatio', 'NearestStation', 'TimeToNearestStation', 'Use', 'PricePerTsubo', 'UnitPrice', 'MunicipalityCode', 'Period', 'DistrictName'], inplace=True)

# Combine MinTimeToNearestStation and MaxTimeToNearestStation into AvgTimeToNearestStation
df_01['AvgTimeToNearestStation'] = df_01[['MinTimeToNearestStation', 'MaxTimeToNearestStation']].mean(axis=1)

df_01.drop(columns=['MinTimeToNearestStation', 'MaxTimeToNearestStation'], inplace=True)

# Convert relevant columns to bool type
bool_columns = ['AreaIsGreaterFlag', 'FrontageIsGreaterFlag', 'TotalFloorAreaIsGreaterFlag', 'PrewarBuilding']

for col in bool_columns:
    df_01[col] = df_01[col].astype(bool)



In [4]:
# We only want to investigate real estate that is intended to be used as a house

#Residential_condition = (df_01["Type"] != "Forest Land") & (df_01["Type"] != "Agricultural Land")
Intended_House_condition = (df_01["Purpose"] == "House")

House_df = df_01[Intended_House_condition]
House_df.drop(columns=['Purpose'], inplace=True) # No longer needed
print(f'{len(House_df)} entries, ({(len(House_df) / len(df_01)) * 100} % of total) satisfy the housing condition.')


42410 entries, (22.771936983859362 % of total) satisfy the housing condition.


/tmp/ipykernel_166860/1275131266.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df.drop(columns=['Purpose'], inplace=True) # No longer needed


In [5]:
# Any type that is Land Only will not have a floor size, so we can set the TotalFloorArea to -1. Same logic for Building year and frontage
House_df.loc[House_df['TotalFloorArea'].isna() & (House_df['Type'] == 'Residential Land(Land Only)'), 'TotalFloorArea'] = -1
House_df.loc[House_df['BuildingYear'].isna() & (House_df['Type'] == 'Residential Land(Land Only)'), 'BuildingYear'] = -1
House_df.loc[House_df['Frontage'].isna() & (House_df['Type'] == 'Residential Land(Land Only)'), 'Frontage'] = -1
#If the region or time to station cannot be determined, delete
# Drop rows where 'Region' or time to station has NaN values
House_df.dropna(subset=['Region', 'AvgTimeToNearestStation'], inplace=True)



/tmp/ipykernel_166860/3888191932.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df.dropna(subset=['Region', 'AvgTimeToNearestStation'], inplace=True)


In [6]:
# Count rows with NaN values in the entire DataFrame
nan_rows_count = House_df.isna().sum(axis=1)
rows_with_nan = nan_rows_count[nan_rows_count > 0]

# Output the number of rows with NaN values
print(f'Number of rows with NaN values: {len(rows_with_nan)}')

Number of rows with NaN values: 2038


In [7]:
#Drop remaining NaN values
House_df.dropna(inplace=True)


/tmp/ipykernel_166860/2211310912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df.dropna(inplace=True)


In [8]:
House_df

,Type,Region,Prefecture,Municipality,TradePrice,Area,AreaIsGreaterFlag,Frontage,FrontageIsGreaterFlag,TotalFloorArea,TotalFloorAreaIsGreaterFlag,BuildingYear,PrewarBuilding,Year,Quarter,AvgTimeToNearestStation
1,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward,Sapporo City",38000000,310,False,21.5,False,-1.0,False,-1.0,False,2018,4,27.0
4,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward,Sapporo City",25000000,430,False,16.0,False,-1.0,False,-1.0,False,2018,2,23.0
8,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward,Sapporo City",20000000,165,False,12.6,False,-1.0,False,-1.0,False,2017,4,29.0
12,Residential Land(Land and Building),Residential Area,Hokkaido,"Chuo Ward,Sapporo City",40000000,400,False,14.0,False,190.0,False,1989.0,False,2017,2,28.0
20,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward,Sapporo City",23000000,370,False,-1.0,False,-1.0,False,-1.0,False,2016,2,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179713,Residential Land(Land and Building),Commercial Area,Hokkaido,"Shiranuka Town,Shiranuka County",1600000,250,False,9.1,False,230.0,False,1989.0,False,2016,3,4.0
179730,Residential Land(Land and Building),Residential Area,Hokkaido,"Shiranuka Town,Shiranuka County",6600000,190,False,10.0,False,65.0,False,2002.0,False,2019,3,4.0
179734,Residential Land(Land and Building),Residential Area,Hokkaido,"Shiranuka Town,Shiranuka County",570000,95,False,6.6,False,145.0,False,1994.0,False,2016,4,2.0
179736,Residential Land(Land and Building),Commercial Area,Hokkaido,"Shiranuka Town,Shiranuka County",2000000,210,False,8.9,False,100.0,False,1991.0,False,2014,4,2.0


In [9]:
type_encoded = pd.get_dummies(House_df['Type'], prefix='Type')
# Merge with original dataframe
House_df = pd.concat([House_df, type_encoded], axis=1)

# Drop the original 'Type' column
#

# Check result
House_df

,Type,Region,Prefecture,Municipality,TradePrice,Area,AreaIsGreaterFlag,Frontage,FrontageIsGreaterFlag,TotalFloorArea,TotalFloorAreaIsGreaterFlag,BuildingYear,PrewarBuilding,Year,Quarter,AvgTimeToNearestStation,Type_Residential Land(Land Only),Type_Residential Land(Land and Building)
1,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward,Sapporo City",38000000,310,False,21.5,False,-1.0,False,-1.0,False,2018,4,27.0,True,False
4,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward,Sapporo City",25000000,430,False,16.0,False,-1.0,False,-1.0,False,2018,2,23.0,True,False
8,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward,Sapporo City",20000000,165,False,12.6,False,-1.0,False,-1.0,False,2017,4,29.0,True,False
12,Residential Land(Land and Building),Residential Area,Hokkaido,"Chuo Ward,Sapporo City",40000000,400,False,14.0,False,190.0,False,1989.0,False,2017,2,28.0,False,True
20,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward,Sapporo City",23000000,370,False,-1.0,False,-1.0,False,-1.0,False,2016,2,24.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179713,Residential Land(Land and Building),Commercial Area,Hokkaido,"Shiranuka Town,Shiranuka County",1600000,250,False,9.1,False,230.0,False,1989.0,False,2016,3,4.0,False,True
179730,Residential Land(Land and Building),Residential Area,Hokkaido,"Shiranuka Town,Shiranuka County",6600000,190,False,10.0,False,65.0,False,2002.0,False,2019,3,4.0,False,True
179734,Residential Land(Land and Building),Residential Area,Hokkaido,"Shiranuka Town,Shiranuka County",570000,95,False,6.6,False,145.0,False,1994.0,False,2016,4,2.0,False,True
179736,Residential Land(Land and Building),Commercial Area,Hokkaido,"Shiranuka Town,Shiranuka County",2000000,210,False,8.9,False,100.0,False,1991.0,False,2014,4,2.0,False,True


In [10]:
House_df['Region'].unique()

region_encoded = pd.get_dummies(House_df['Region'], prefix='Region')
# Merge with original dataframe
House_df = pd.concat([House_df, region_encoded], axis=1)

# Drop the original 'Type' column
House_df.drop(columns=['Region'], inplace=True)

# Check result
House_df

,Type,Prefecture,Municipality,TradePrice,Area,AreaIsGreaterFlag,Frontage,FrontageIsGreaterFlag,TotalFloorArea,TotalFloorAreaIsGreaterFlag,...,PrewarBuilding,Year,Quarter,AvgTimeToNearestStation,Type_Residential Land(Land Only),Type_Residential Land(Land and Building),Region_Commercial Area,Region_Industrial Area,Region_Potential Residential Area,Region_Residential Area
1,Residential Land(Land Only),Hokkaido,"Chuo Ward,Sapporo City",38000000,310,False,21.5,False,-1.0,False,...,False,2018,4,27.0,True,False,False,False,False,True
4,Residential Land(Land Only),Hokkaido,"Chuo Ward,Sapporo City",25000000,430,False,16.0,False,-1.0,False,...,False,2018,2,23.0,True,False,False,False,False,True
8,Residential Land(Land Only),Hokkaido,"Chuo Ward,Sapporo City",20000000,165,False,12.6,False,-1.0,False,...,False,2017,4,29.0,True,False,False,False,False,True
12,Residential Land(Land and Building),Hokkaido,"Chuo Ward,Sapporo City",40000000,400,False,14.0,False,190.0,False,...,False,2017,2,28.0,False,True,False,False,False,True
20,Residential Land(Land Only),Hokkaido,"Chuo Ward,Sapporo City",23000000,370,False,-1.0,False,-1.0,False,...,False,2016,2,24.0,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179713,Residential Land(Land and Building),Hokkaido,"Shiranuka Town,Shiranuka County",1600000,250,False,9.1,False,230.0,False,...,False,2016,3,4.0,False,True,True,False,False,False
179730,Residential Land(Land and Building),Hokkaido,"Shiranuka Town,Shiranuka County",6600000,190,False,10.0,False,65.0,False,...,False,2019,3,4.0,False,True,False,False,False,True
179734,Residential Land(Land and Building),Hokkaido,"Shiranuka Town,Shiranuka County",570000,95,False,6.6,False,145.0,False,...,False,2016,4,2.0,False,True,False,False,False,True
179736,Residential Land(Land and Building),Hokkaido,"Shiranuka Town,Shiranuka County",2000000,210,False,8.9,False,100.0,False,...,False,2014,4,2.0,False,True,True,False,False,False


In [11]:
House_df['Prefecture'].replace('Hokkaido', 0, inplace=True)
House_df

/tmp/ipykernel_166860/1432698123.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  House_df['Prefecture'].replace('Hokkaido', 0, inplace=True)
/tmp/ipykernel_166860/1432698123.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  House_df['Prefecture'].replace('Hokkaido', 0, inplace=True)


,Type,Prefecture,Municipality,TradePrice,Area,AreaIsGreaterFlag,Frontage,FrontageIsGreaterFlag,TotalFloorArea,TotalFloorAreaIsGreaterFlag,...,PrewarBuilding,Year,Quarter,AvgTimeToNearestStation,Type_Residential Land(Land Only),Type_Residential Land(Land and Building),Region_Commercial Area,Region_Industrial Area,Region_Potential Residential Area,Region_Residential Area
1,Residential Land(Land Only),0,"Chuo Ward,Sapporo City",38000000,310,False,21.5,False,-1.0,False,...,False,2018,4,27.0,True,False,False,False,False,True
4,Residential Land(Land Only),0,"Chuo Ward,Sapporo City",25000000,430,False,16.0,False,-1.0,False,...,False,2018,2,23.0,True,False,False,False,False,True
8,Residential Land(Land Only),0,"Chuo Ward,Sapporo City",20000000,165,False,12.6,False,-1.0,False,...,False,2017,4,29.0,True,False,False,False,False,True
12,Residential Land(Land and Building),0,"Chuo Ward,Sapporo City",40000000,400,False,14.0,False,190.0,False,...,False,2017,2,28.0,False,True,False,False,False,True
20,Residential Land(Land Only),0,"Chuo Ward,Sapporo City",23000000,370,False,-1.0,False,-1.0,False,...,False,2016,2,24.0,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179713,Residential Land(Land and Building),0,"Shiranuka Town,Shiranuka County",1600000,250,False,9.1,False,230.0,False,...,False,2016,3,4.0,False,True,True,False,False,False
179730,Residential Land(Land and Building),0,"Shiranuka Town,Shiranuka County",6600000,190,False,10.0,False,65.0,False,...,False,2019,3,4.0,False,True,False,False,False,True
179734,Residential Land(Land and Building),0,"Shiranuka Town,Shiranuka County",570000,95,False,6.6,False,145.0,False,...,False,2016,4,2.0,False,True,False,False,False,True
179736,Residential Land(Land and Building),0,"Shiranuka Town,Shiranuka County",2000000,210,False,8.9,False,100.0,False,...,False,2014,4,2.0,False,True,True,False,False,False


It is difficult to deal with so much categorical data. 

There are 47 prefectures. Hokkaido, has approximately 140 muncipalities.

I think my approach should be like this:

For the 47 prefectures: One-Hot Encode the prefectures into the 8 blocks: Hokkaidō (the island and its archipelago)

- Hokkaido = 0
- Tōhoku region (northern part)
- Kantō region (eastern part)
- Chūbu region (central part)
- Kansai (or Kinki) region (south-central part)
- Chūgoku region (western part)
- Shikoku
- Kyūshū = 8

For the Muncipalities, I could maybe categorise them as "Capital", "City", "Town" and "Village"

In [12]:
capital_dict = {0:"Sapporo"}
def categorize_municipality(municipality, prefecture):
    if 'Sapporo' in municipality:
        return 4  # Capital
    elif 'City' in municipality:
        return 3  # City
    elif 'Town' in municipality:
        return 2  # Town
    elif 'Village' in municipality:
        return 1  # Village
    else:
        return 0  # Default

# Apply the categorization function
House_df['MunicipalityCategory'] = House_df.apply(lambda row: categorize_municipality(row['Municipality'], row['Prefecture']), axis=1)

In [13]:
House_df.drop(columns=['Municipality'], inplace=True)
House_df

,Type,Prefecture,TradePrice,Area,AreaIsGreaterFlag,Frontage,FrontageIsGreaterFlag,TotalFloorArea,TotalFloorAreaIsGreaterFlag,BuildingYear,...,Year,Quarter,AvgTimeToNearestStation,Type_Residential Land(Land Only),Type_Residential Land(Land and Building),Region_Commercial Area,Region_Industrial Area,Region_Potential Residential Area,Region_Residential Area,MunicipalityCategory
1,Residential Land(Land Only),0,38000000,310,False,21.5,False,-1.0,False,-1.0,...,2018,4,27.0,True,False,False,False,False,True,4
4,Residential Land(Land Only),0,25000000,430,False,16.0,False,-1.0,False,-1.0,...,2018,2,23.0,True,False,False,False,False,True,4
8,Residential Land(Land Only),0,20000000,165,False,12.6,False,-1.0,False,-1.0,...,2017,4,29.0,True,False,False,False,False,True,4
12,Residential Land(Land and Building),0,40000000,400,False,14.0,False,190.0,False,1989.0,...,2017,2,28.0,False,True,False,False,False,True,4
20,Residential Land(Land Only),0,23000000,370,False,-1.0,False,-1.0,False,-1.0,...,2016,2,24.0,True,False,False,False,False,True,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179713,Residential Land(Land and Building),0,1600000,250,False,9.1,False,230.0,False,1989.0,...,2016,3,4.0,False,True,True,False,False,False,2
179730,Residential Land(Land and Building),0,6600000,190,False,10.0,False,65.0,False,2002.0,...,2019,3,4.0,False,True,False,False,False,True,2
179734,Residential Land(Land and Building),0,570000,95,False,6.6,False,145.0,False,1994.0,...,2016,4,2.0,False,True,False,False,False,True,2
179736,Residential Land(Land and Building),0,2000000,210,False,8.9,False,100.0,False,1991.0,...,2014,4,2.0,False,True,True,False,False,False,2


In [ ]:
# Create a Normalization layer
#normalizer = tf.keras.layers.Normalization()

# Fit the normalizer on your data (e.g., on numerical columns)
#normalizer.adapt(House_df[['TradePrice']].values)

# Apply normalization to the data
#normalized_data = normalizer(House_df[['TradePrice']].values)

# Convert the normalized data back into a DataFrame and assign it to House_df
#normalized_df = pd.DataFrame(normalized_data.numpy(), columns=['TradePrice'])

# Replace the original columns with the normalized data
#House_df[['TradePrice']] = normalized_df

# Check the result
#House_df

In [16]:
# Filter the dataset for properties with buildings and those without buildings
land_only_df = House_df[House_df['Type_Residential Land(Land Only)'] == True]
building_df = House_df[House_df['Type_Residential Land(Land Only)'] == False]

# Save the filtered datasets to CSV files
land_only_df.to_csv('Hokkaido_cleaned_test_landOnly.csv', index=False)
building_df.to_csv('Hokkaido_cleaned_test_buildings.csv', index=False)
